**Inteligência Artificial**

**812839 - Vinícius Miranda de Araújo**

**Lista de Exercícios 13**

---
---

## Questão 1

### Instalando  e importando bibliotecas

In [ ]:
!pip install scikit-multilearn
!pip install scikit-learn
!pip install nltk
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, hamming_loss, classification_report
from skmultilearn.problem_transform import BinaryRelevance
from nltk.corpus import stopwords
import nltk
from imblearn.over_sampling import RandomOverSampler
from scipy.sparse import csr_matrix
from scipy.sparse import vstack

### Carregando a base de dados

In [ ]:
nltk.download('stopwords')

# Carregar o dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_labels = pd.read_csv('test_labels.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Visualizando
print("Dados de treino:\n")
print(train.head())
print(train.shape)

print("\nDados de teste:\n")
print(test.head())
print(test.shape)

print("\nLabels de teste:\n")
print(test_labels.head())
print(test_labels.shape)

Dados de treino:

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  
(159571, 8)

Dados de teste:

                 id                                       comment_text
0  00001cee341fdb12  Yo bitch Ja Rule is more 

### Pré-processamento do texto

In [ ]:
# Labels de classificação
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
# Pré-processamento dos textos
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove pontuação
    text = re.sub(r'\d+', '', text)      # Remove números
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

# Aplicar pré-processamento
train['comment_text'] = train['comment_text'].apply(preprocess)
test['comment_text'] = test['comment_text'].apply(preprocess)

# Vetorização TF-IDF
tfidf = TfidfVectorizer(max_features=10000)
X_train = tfidf.fit_transform(train['comment_text'])
X_test = tfidf.transform(test['comment_text'])

y_train = train[labels].values
y_test_labels = test_labels[labels].values

# Filtrar apenas os exemplos válidos no conjunto de teste
# (Aqueles que não têm -1 em nenhuma das labels)
valid_rows = ~(y_test_labels == -1).any(axis=1)

X_test_valid = X_test[valid_rows]
y_test_valid = y_test_labels[valid_rows]

### Modelo 1: Sem balanceamento

In [ ]:
model_base = BinaryRelevance(classifier=LogisticRegression(max_iter=1000))
model_base.fit(X_train, y_train)
y_pred_base = model_base.predict(X_test_valid)

#### Avaliação do Modelo 1

In [ ]:
print("\n=== Avaliação Modelo Base ===")
print("Hamming Loss:", hamming_loss(y_test_valid, y_pred_base))
print("F1 Micro:", f1_score(y_test_valid, y_pred_base, average='micro'))
print("F1 Macro:", f1_score(y_test_valid, y_pred_base, average='macro'))
print("Subset Accuracy:", accuracy_score(y_test_valid, y_pred_base))
print(classification_report(y_test_valid, y_pred_base, target_names=labels))


=== Avaliação Modelo Base ===
Hamming Loss: 0.025782300165681952
F1 Micro: 0.6370071520264075
F1 Macro: 0.4871900326087752
Subset Accuracy: 0.8951201975679139
               precision    recall  f1-score   support

        toxic       0.65      0.71      0.68      6090
 severe_toxic       0.41      0.32      0.36       367
      obscene       0.75      0.61      0.68      3691
       threat       0.45      0.18      0.26       211
       insult       0.73      0.52      0.61      3427
identity_hate       0.62      0.24      0.35       712

    micro avg       0.68      0.60      0.64     14498
    macro avg       0.60      0.43      0.49     14498
 weighted avg       0.68      0.60      0.63     14498
  samples avg       0.06      0.06      0.06     14498



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Modelo 2: Com Balanceamento (Oversampling)

In [ ]:
# Listas para armazenar dados balanceados temporariamente
X_list = []
y_list = []

for i, label in enumerate(labels):
    print(f"\nBalanceando label: {label}")

    # Extrai o rótulo específico
    y_label = y_train[:, i]

    # Inicializa o oversampler
    ros = RandomOverSampler(random_state=42)

    # Aplica o oversampling para essa label específica
    X_res, y_res = ros.fit_resample(X_train, y_label)

    # Cria o vetor de labels multilabel com zeros
    y_res_multi = np.zeros((X_res.shape[0], len(labels)), dtype=int)

    # Preenche apenas a coluna da label atual com os valores balanceados
    y_res_multi[:, i] = y_res

    # Armazena os dados
    X_list.append(X_res)
    y_list.append(y_res_multi)

# Concatena todos os X e Y das labels balanceadas
X_bal = vstack(X_list)
y_bal = np.vstack(y_list)

# Remove duplicatas
# df_X = pd.DataFrame(X_bal.toarray())
# df_y = pd.DataFrame(y_bal, columns=labels)
# df_concat = pd.concat([df_X, df_y], axis=1).drop_duplicates()

# Separa novamente X e y
# X_bal = csr_matrix(df_concat.iloc[:, :-len(labels)].values)
# y_bal = df_concat.iloc[:, -len(labels):].values

print("\nShape final após balanceamento e remoção de duplicatas:", X_bal.shape, y_bal.shape)

In [ ]:
# Modelo balanceado
model_balanced = BinaryRelevance(classifier=LogisticRegression(max_iter=1000))
model_balanced.fit(X_bal, y_bal)

# Predição no conjunto de teste
y_pred_balanced = model_balanced.predict(X_test_valid)

#### Avaliação do Modelo 2

In [ ]:
print("\n=== Avaliação Modelo Balanceado ===")
print("Hamming Loss:", hamming_loss(y_test_valid, y_pred_balanced))
print("F1 Micro:", f1_score(y_test_valid, y_pred_balanced, average='micro'))
print("F1 Macro:", f1_score(y_test_valid, y_pred_balanced, average='macro'))
print("Subset Accuracy:", accuracy_score(y_test_valid, y_pred_balanced))
print(classification_report(y_test_valid, y_pred_balanced, target_names=labels))

## Questão 2

| Questão | Resposta | Justificativa                                                                                                                                                                                                        |
| ------- | -------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1       | A        | A alternativa apresenta corretamente a contrapositiva da sentença "Se é feriado, os bancos estão fechados", que é "Se os bancos não estão fechados, então não é feriado". Isso utiliza a **regra de contraposição**. |
| 4       | E        | Utilizando o **silogismo disjuntivo**, se uma das alternativas "não chover" leva Cláudia à praia e "chover" leva Fábia ao clube, então, independentemente do tempo, uma das duas irá se divertir.                    |
| 5       | A        | Aplicação do **modus ponens**: se as premissas são verdadeiras, então a conclusão também é verdadeira.                                                                                                               |
| 7       | A        | Uso do **silogismo hipotético**, onde se $A$ implica $B$ e $B$ implica $C$, então $A$ implica $C$.                                                                                                                   |
| 8       | B        | Se não vou à escola, então não há aula. Pela **contraposição**, se há aula, então vou à escola.                                                                                                                      |
| 9       | E        | A alternativa correta aplica a regra do **silogismo disjuntivo**, eliminando uma opção para concluir a outra                                                                                                         |
| 13      | C        | A alternativa faz uso da **regra de contraposição**, que é logicamente equivalente à condicional.                                                                                                                    |
| 16      | E        | Pelo **silogismo disjuntivo**, ao negar uma das alternativas de uma disjunção verdadeira, conclui-se que a outra é verdadeira.                                                                                       |
| 17      | B        | Uso da **adição**, que permite que, de uma proposição simples, se derive uma disjunção verdadeira.                                                                                                                   |
| 19      | A        | Aplicação direta da regra do **modus ponens**: se $p \to q$ e $p$ é verdadeiro, então $q$ também é.                                                                                                                  |
| 20      | B        | Pela **contraposição**, a condicional "Se $p$ então $q$" é equivalente a "Se não $q$ então não $p$".                                                                                                                 |
| 21      | C        | Aplicação do **silogismo disjuntivo**, descartando uma opção para concluir a outra.                                                                                                                                  |
| 23      | B        | Uso da **contraposição**, transformando "Se $p$ então $q$" em "Se não $q$ então não $p$".                                                                                                                            |
| 24      | C        | Uso da **adição**, que permite criar uma disjunção a partir de uma única proposição verdadeira.                                                                                                                      |
| 26      | D        | Aplicação da regra de **contraposição**, reconhecendo a equivalência lógica entre uma condicional e sua contrapositiva.                                                                                              |
| 27      | C        | Pela **exportação**, uma condicional com conjunção no antecedente pode ser reescrita como uma condicional encadeada.                                                                                                 |
| 28      | A        | Aplicação do **modus tollens**: se $p \to q$ e $~q$, então $~p$.                                                                                                                                                     |
| 31      | E        | Não se pode concluir diretamente nada sobre a culpa de Francisco, apenas que não desviou dinheiro. A alternativa E é a que resta como logicamente correta, mas sem informação específica.                            |
| 32      | A        | Aplicação da **contraposição**: "Se Rodrigo mentiu então ele é culpado" é equivalente a "Se ele não é culpado então ele não mentiu".                                                                                 |


## Questão 3

### a) Aplicação da regra clássica:

> “Se a altura $\ge$ 170 cm, então é alto (1); senão, não é alto (0)”.

| Personagem | Altura (cm) | Alto (lógica clássica) |
| ---------- | ----------- | ---------------------- |
| Ana        | 148         | 0                      |
| Bruno      | 165         | 0                      |
| Carla      | 172         | 1                      |
| Diego      | 180         | 1                      |
| Elisa      | 191         | 1                      |

### b) Discutindo se faz sentido dizer que alguém com 169 cm não é nada alto (0) e alguém com 170 cm é totalmente alto (1)?

Não faz sentido absoluto. A diferença de 1 cm (169 para 170) não representa uma mudança drástica no conceito de "alto". Por isso, a lógica fuzzy é mais adequada, pois permite graus de pertencimento.

### c) Propondo uma função de pertinência fuzzy para a categoria "alto", que varie de 0 (não é alto) até 1 (muito alto), usando a seguinte lógica linear aproximada:

- Se altura < 160 $\to$ grau = 0
    
- Se altura > 190 $\to$ grau = 1
    
- Se altura entre 160 e 190 $\to$ $\text{grau} = \frac{(\text{altura} - 160)}{(190 - 160)}$

Ou seja:

$$\text{grau} = \frac{\text{altura} - 160}{30}$$


### d) Preenchendo a tabela com os valores fuzzy de "alto":

| Personagem | Altura (cm) | Grau de "alto" (fuzzy)                   |
| ---------- | ----------- | ---------------------------------------- |
| Ana        | 148         | 0                                        |
| Bruno      | 165         | $\frac{(165-160)}{30}$ = 0,166... ≈ 0,17 |
| Carla      | 172         | $\frac{(172-160)}{30}$ = 0,4             |
| Diego      | 180         | $\frac{(180-160)}{30}$ = 0,666... ≈ 0,67 |
| Elisa      | 191         | 1                                        |